# Tutorial: Programmatic Simulation and Control

This notebook demonstrates how to use the CHS SDK as a Python library to programmatically build and run a simulation. Unlike using YAML files, this approach offers maximum flexibility and is ideal for research, complex scripting, and integration with other Python libraries.

**Goal:** We will create a simple closed-loop control system consisting of:
1.  A **Reservoir**, represented by a `BodyAgent` with a `LinearTank` model.
2.  A **PID Controller**, represented by a `ControlAgent`.
3.  A **Logger**, a special agent that will record the reservoir's state for later analysis.

We will use the new state-reporting mechanism: the reservoir will report its state, and the PID controller will subscribe to these updates to get its feedback signal.

## 1. Imports and Setup

First, we need to import all the necessary classes from the SDK. We also import `matplotlib` for plotting.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd

# Add the project root to the path to allow imports from chs_sdk
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Core SDK components
from chs_sdk.core.host import AgentKernel
from chs_sdk.agents.body_agent import BodyAgent
from chs_sdk.agents.control_agent import ControlAgent

# Helper/Utility agents for the tutorial
from tests.helpers.mock_agents import LoggerAgent

# Scientific models
from chs_sdk.modules.modeling.storage_models import LinearTank

## 2. Initialize the Simulation Kernel

The `AgentKernel` is the heart of the simulation. It manages all the agents and the message bus for their communication.

In [ ]:
kernel = AgentKernel()

## 3. Create and Configure the Agents

Now, we'll create our three agents: the reservoir (BodyAgent), the controller (ControlAgent), and the logger (LoggerAgent).

In [ ]:
# Define agent IDs for clarity
reservoir_id = "reservoir_1"
controller_id = "pid_controller_1"
logger_id = "logger_1"

# Create the physical model for the reservoir
reservoir_model = LinearTank(area=1000, initial_level=5.0)

# 1. Create the BodyAgent for the reservoir
kernel.add_agent(
    BodyAgent,
    agent_id=reservoir_id,
    core_physics_model=reservoir_model,
    sensors={},
    actuators={}
)

# 2. Create the ControlAgent for the PID controller
kernel.add_agent(
    ControlAgent,
    agent_id=controller_id,
    # Configuration for this agent:
    target_agent_id=reservoir_id,         # Which agent to control
    variable_to_control="level",        # Which variable in the state to use as feedback
    control_algorithm="PID",
    algorithm_config={"Kp": 2.0, "Ki": 0.1, "Kd": 0.0, "set_point": 10.0} # Target level is 10.0m
)

# 3. Create the LoggerAgent to record the reservoir's state
kernel.add_agent(
    LoggerAgent,
    agent_id=logger_id,
    # Configuration for this agent:
    topic_to_subscribe=f"state/update/{reservoir_id}"
)

# We also need to connect the controller's output to the reservoir's input.
# The new architecture doesn't have a standard message for this yet, so we will do it manually in the loop.

## 4. Running the Simulation

With the agents created, we can now run the simulation. We will manually step through the simulation to show how the controller's output can be fed back to the reservoir.

In [ ]:
reservoir_agent = kernel._agents[reservoir_id]
pid_agent = kernel._agents[controller_id]
logger_agent = kernel._agents[logger_id]

duration = 200  # seconds
time_step = 1.0 # second

# Manually set up the kernel for ticking
kernel.start(time_step=time_step)

for t in range(int(duration / time_step)):
    # In each step, we manually set the reservoir's inflow to be the PID's last command
    # This closes the control loop.
    if pid_agent.current_command is not None:
        reservoir_agent.core_physics_model.input.inflow = pid_agent.current_command
    
    kernel.tick()

# Shut down the kernel
kernel.stop()

## 5. Analyzing and Visualizing the Results

Now that the simulation is complete, we can access the data captured by our `LoggerAgent` and plot it to see if our controller worked.

In [ ]:
# The logger's history contains a list of state dictionaries
history = logger_agent.history

# Convert the history to a pandas DataFrame for easy analysis
df = pd.DataFrame(history)

# Add a time column
df['time'] = [i * time_step for i in range(len(df))]

print("First 5 logged states:")
print(df.head())

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(df['time'], df['level'], label='Reservoir Water Level')
plt.axhline(y=10.0, color='r', linestyle='--', label='Setpoint (10.0m)')
plt.title('PID Control of Reservoir Water Level')
plt.xlabel('Time (s)')
plt.ylabel('Water Level (m)')
plt.legend()
plt.grid(True)
plt.show()

### Conclusion

As you can see from the plot, the PID controller successfully brings the reservoir's water level from its initial state of 5.0m up to the target setpoint of 10.0m. 

This notebook has demonstrated:
- How to create and configure agents programmatically.
- How the state-reporting mechanism works in practice.
- How to run the `AgentKernel` step-by-step.
- How to create a simple `LoggerAgent` to capture results.
- How to analyze and plot the simulation output.

You can use this template to build much more complex simulations and control strategies.